In [20]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline



pd.set_option('display.max_columns', None)

In [2]:
file= pd.read_csv('Customer-Churn-Records.csv')

In [3]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

In [4]:
file.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [5]:
df_cln=file.drop(columns=['CustomerId','Surname'])

In [6]:
df_customer_id=file[['CustomerId','Surname']]

In [7]:
df_customer_id

,CustomerId,Surname
0,15634602,Hargrave
1,15647311,Hill
2,15619304,Onio
3,15701354,Boni
4,15737888,Mitchell
...,...,...
9995,15606229,Obijiaku
9996,15569892,Johnstone
9997,15584532,Liu
9998,15682355,Sabbatini


In [8]:
df_cln=df_cln.drop('RowNumber', axis=1)

In [9]:
df_cln

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0,1,DIAMOND,300
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0,5,PLATINUM,771
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1,1,3,SILVER,564
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,1,2,GOLD,339


In [10]:
df_cln=pd.get_dummies(df_cln, columns=['Geography', 'Gender', 'Card Type']).astype(int)

In [11]:
df_cln

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Card Type_DIAMOND,Card Type_GOLD,Card Type_PLATINUM,Card Type_SILVER
0,619,42,2,0,1,1,1,101348,1,1,2,464,1,0,0,1,0,1,0,0,0
1,608,41,1,83807,1,0,1,112542,0,1,3,456,0,0,1,1,0,1,0,0,0
2,502,42,8,159660,3,1,0,113931,1,1,3,377,1,0,0,1,0,1,0,0,0
3,699,39,1,0,2,0,0,93826,0,0,5,350,1,0,0,1,0,0,1,0,0
4,850,43,2,125510,1,1,1,79084,0,0,5,425,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0,2,1,0,96270,0,0,1,300,1,0,0,0,1,1,0,0,0
9996,516,35,10,57369,1,1,1,101699,0,0,5,771,1,0,0,0,1,0,0,1,0
9997,709,36,7,0,1,0,1,42085,1,1,3,564,1,0,0,1,0,0,0,0,1
9998,772,42,3,75075,2,1,0,92888,1,1,2,339,0,1,0,0,1,0,1,0,0


In [12]:
df_cln=df_cln.astype(int)

In [13]:
df_cln

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Card Type_DIAMOND,Card Type_GOLD,Card Type_PLATINUM,Card Type_SILVER
0,619,42,2,0,1,1,1,101348,1,1,2,464,1,0,0,1,0,1,0,0,0
1,608,41,1,83807,1,0,1,112542,0,1,3,456,0,0,1,1,0,1,0,0,0
2,502,42,8,159660,3,1,0,113931,1,1,3,377,1,0,0,1,0,1,0,0,0
3,699,39,1,0,2,0,0,93826,0,0,5,350,1,0,0,1,0,0,1,0,0
4,850,43,2,125510,1,1,1,79084,0,0,5,425,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0,2,1,0,96270,0,0,1,300,1,0,0,0,1,1,0,0,0
9996,516,35,10,57369,1,1,1,101699,0,0,5,771,1,0,0,0,1,0,0,1,0
9997,709,36,7,0,1,0,1,42085,1,1,3,564,1,0,0,1,0,0,0,0,1
9998,772,42,3,75075,2,1,0,92888,1,1,2,339,0,1,0,0,1,0,1,0,0


In [14]:
X=df_cln.drop('Exited', axis=1)
y=df_cln['Exited']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [16]:
std=StandardScaler()
# X_train_scaled=std.fit_transform(X_train)
# X_test_scaled=std.transform(X_test)

# First Balance with SMOTE

In [18]:
sm = SMOTE(random_state=42)
X_tr_bal, y_tr_bal = sm.fit_resample(X_train_scaled, y_train)

In [ ]:
pipe= [
    ('scaler',std)
    ('classifier',RandomForestClassifier())
]